In [ ]:
import pyvisa as visa
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
rm = visa.ResourceManager()
print(rm.list_resources('?*'))

fgen = rm.open_resource('USB0::62700::4353::SDG6XEBX4R0234::0::INSTR')


In [ ]:
#set up

#set burst mode
fgen.write("C1:BTWV STATE,ON")
#set gate mode and lenght of gate
fgen.write("C1:BTWV GATE_NCYC,GATE")
fgen.write("C1:BTWV PRD,.01") #time in seconds to gate
#set polarity
fgen.write("C1:BTWV PLRT,NEG")
#set initial frequency
fgen.write("C1:BTWV CARR,FRQ,200000000")

#set pulse wave as trigger on ch2
fgen.write("C2:BSWV WVTP,PULSE") #pulse
fgen.write("C2:BSWV RISE,.000000001") #sets 1ns rise time (ASAP)
#set trigger to 0-1V
fgen.write("C2:BSWV AMP,1") 
fgen.write("C2:BSWV OFST,.5")
fgen.write("C2:BSWV DLY,-.0000003") #delay time (pretrigger)
fgen.write("C2:BTWV CARR,FRQ,100")


In [ ]:
freq = 30.0001e6 #Hz
freqs = np.linspace(30e6,500e6,10)
for freq in freqs:
    fgen.write("C1:BTWV CARR,FRQ,"+str(int(freq)))
    time.sleep(1)

In [ ]:
#!/usr/bin/env python3
# Copyright 2022 Teledyne Signal Processing Devices Sweden AB
"""
 Example illustrating the data acquisition for a ADQ3 series digitizer. The
 example will list the available devices and acquire data from the first
 device.
"""

import pyadq
from typing import List

# Record length in samples
RECORD_LENGTH = 2**21
# Number of records to collect per channel
NOF_RECORDS = 1000


SIZEOF_INT16 = 2
NOF_RECORDS_PER_BUFFER = 1

nof_channels = 1

# Create the control unit
acu: pyadq.ADQControlUnit = pyadq.ADQControlUnit()

# Enable trace logging
acu.ADQControlUnit_EnableErrorTrace(pyadq.LOG_LEVEL_INFO, ".")

# List the available devices
device_list: List[pyadq.ADQInfoListEntry] = acu.ListDevices()

print(f"Found {len(device_list)} device(s)")

# Ensure that at least one device is available
assert device_list

# Set up the first available device
device_to_open = 0
dev: pyadq.ADQ = acu.SetupDevice(device_to_open)

print(f"Setting up data collection for: {dev}")

# Initialize the parameters with default values
parameters: pyadq.ADQParameters = dev.InitializeParameters(pyadq.ADQ_PARAMETER_ID_TOP)

# Set up data collection for all channels
for acq_param in parameters.acquisition.channel[0 : nof_channels]:
    acq_param.record_length = RECORD_LENGTH
    acq_param.nof_records = pyadq.ADQ_INFINITE_NOF_RECORDS
    acq_param.trigger_edge = pyadq.ADQ_EDGE_RISING
    acq_param.trigger_source = pyadq.ADQ_EVENT_SOURCE_TRIG

# Configure data transfer parameters for all channels
for transfer_param in parameters.transfer.channel[0 : nof_channels]:
    transfer_param.record_size = SIZEOF_INT16 * RECORD_LENGTH
    transfer_param.record_length_infinite_enabled = 0
    transfer_param.record_buffer_size = (
        NOF_RECORDS_PER_BUFFER * SIZEOF_INT16 * RECORD_LENGTH
    )
    # Enable metadata (record headers)
    transfer_param.metadata_enabled = 1
    transfer_param.metadata_buffer_size = (
        NOF_RECORDS_PER_BUFFER * pyadq.SIZEOF_ADQ_GEN4_HEADER
    )
    transfer_param.nof_buffers = pyadq.ADQ_MAX_NOF_BUFFERS


# Configure the periodic event generator
parameters.event_source.periodic.frequency = PERIODIC_EVENT_GENERATOR_FREQUENCY


# Set parameters
dev.SetParameters(parameters)

# Start the data acquisition
print("Starting data acquisition")
result = dev.ADQ_StartDataAcquisition()
if result != pyadq.ADQ_EOK:
    raise Exception(
        f"ADQ_StartDataAcquisition failed with error code {result}. See log file."
    )

record_count = 0
records_to_collect = NOF_RECORDS * nof_channels
try:
    while record_count < records_to_collect:
        # Wait for a record buffer on any channel with 1000 ms timeout
        record_buffer = dev.WaitForRecordBuffer(pyadq.ADQ_ANY_CHANNEL, 1000)

        # Print some header information
        if 1:
            print(
                f"S/N: {record_buffer.header.serial_number}, "
                f"channel {record_buffer.header.channel}"
            )
            print(f"\tRecord number: {record_buffer.header.record_number}")
            print(f"\tRecord length: {record_buffer.header.record_length}")
            print(f"\tData: {record_buffer.data}")
        record_buffer.
        record_count += 1
        time.sleep(0)

except Exception as e:
    dev.ADQ_StopDataAcquisition()
    raise e

# Stop the data acquisition
print("Stopping data acquisition")
result = dev.ADQ_StopDataAcquisition()
if result not in [pyadq.ADQ_EOK, pyadq.ADQ_EINTERRUPTED]:
    raise Exception(
        f"ADQ_StartDataAcquisition failed with error code {result}. See log file."
    )


In [ ]:
%matplotlib widget
plt.plot(record_buffer.data)